### Read the feature engineered data

In [1]:
import pandas as pd
import os
import random
filename = r'C:/Users/endwy/Documents/Columbia MSBA/Spring 2019/E4650 - Business Analytics/Project - Kickstarter Analysis/raw_data/df_kickstarterFE.csv'

p = 1  # x% of the lines
# keep the header, then take only x% of lines
# if random from [0,1] interval is greater than p the row will be skipped
df = pd.read_csv(filename, header=0,  skiprows=lambda i: i>0 and random.random() > p)
len(df)
#df=df[:1000]
#len(df)
df.head()

,Unnamed: 0,Launch_deadline_days,Cat_Name,country,staff_pick,state,backers_count,blurb,goal,launched_at,usd_pledged,deadline,name
0,0,30,0,0,0,0,108,"In the year 2025, a young man's superintellige...",4000.0,0,5265.820000,0,A Date in 2025 - A sci-fi comedy short film
1,1,44,1,0,1,0,37,Replacement Place needs your support to combat...,5000.0,1,5225.000000,1,Replacement Place | New York Live Arts
2,2,14,2,0,0,0,27,Pumpkin Spice Love - A Pumpkin Spice 4 Life 30...,250.0,0,362.000000,2,Pumpkin Spice Love!
3,3,30,3,0,0,0,184,Single mama and her 4-year old sail with all-f...,15000.0,2,15678.000000,3,Pirate Mama
4,4,30,4,1,0,0,169,SparKit wishes to produce kit-set Wimshurst Ma...,2500.0,3,7978.645365,4,SparKit - Miniature Electrostatic Generator


In [2]:
df1 = df.drop(['Unnamed: 0','blurb','name','backers_count'],axis=1)

In [3]:
import numpy as np
df['usd_pledged'] = np.log(1+df['usd_pledged'])
df['goal'] = np.log(1+df['goal'])

## Splitting data into train and test

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df1, test_size = 0.3)
X_train = train.drop(['state'],axis=1)
y_train = train['state']
X_test = test.drop(['state'],axis=1)
y_test = test['state']

## XgBoost Model

In [5]:
#!pip install xgboost
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import RandomizedSearchCV # Model evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [27]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],

        'subsample': [0.2,0.4,0.6, 0.8, 1.0],
    
    
        'learning_rate': [0.1,0.08,0.05, 0.03],

        'colsample_bytree': [0.6, 0.8, 1.0],
    
        'colsample_bylevel': [0.6, 0.8, 1.0],
    
        'max_depth': [3, 4, 5,6],
        
        'n_estimators': [100,200,300,400,500],
       
        }
model = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',silent=True, nthread=1)
from sklearn.model_selection import StratifiedKFold
folds = 10 #/**try 15 or 20*/

param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, scoring='roc_auc', cv= 5, verbose=3, random_state=1001 )

 

random_search.fit(X_train , y_train , eval_set = [(X_test,y_test)] ,eval_metric = 'auc')

 

xgbest = random_search.best_estimator_      # This captures the best set of parameters from Randomized search



xgbest.fit(X_train , y_train , eval_set = [(X_test,y_test)] ,eval_metric = 'auc',early_stopping_rounds = 10)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=0.8, n_estimators=100, min_child_weight=5, max_depth=6, learning_rate=0.08, colsample_bytree=1.0, colsample_bylevel=0.6 
[0]	validation_0-auc:0.958482
[1]	validation_0-auc:0.992593
[2]	validation_0-auc:0.995706
[3]	validation_0-auc:0.995912
[4]	validation_0-auc:0.995344
[5]	validation_0-auc:0.99545
[6]	validation_0-auc:0.996556
[7]	validation_0-auc:0.996458
[8]	validation_0-auc:0.996842
[9]	validation_0-auc:0.996894
[10]	validation_0-auc:0.997385
[11]	validation_0-auc:0.997622
[12]	validation_0-auc:0.997644
[13]	validation_0-auc:0.997774
[14]	validation_0-auc:0.997778
[15]	validation_0-auc:0.997843
[16]	validation_0-auc:0.998057
[17]	validation_0-auc:0.998094
[18]	validation_0-auc:0.998115
[19]	validation_0-auc:0.998236
[20]	validation_0-auc:0.998271
[21]	validation_0-auc:0.998324
[22]	validation_0-auc:0.998405
[23]	validation_0-auc:0.998569
[24]	validation_0-auc:0.998582
[25]	validation_0-auc:0.998717
[26]	vali

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.2s remaining:    0.0s


[0]	validation_0-auc:0.958483
[1]	validation_0-auc:0.99198
[2]	validation_0-auc:0.995112
[3]	validation_0-auc:0.99544
[4]	validation_0-auc:0.994725
[5]	validation_0-auc:0.994877
[6]	validation_0-auc:0.996316
[7]	validation_0-auc:0.996212
[8]	validation_0-auc:0.996666
[9]	validation_0-auc:0.996731
[10]	validation_0-auc:0.997233
[11]	validation_0-auc:0.99749
[12]	validation_0-auc:0.997536
[13]	validation_0-auc:0.997697
[14]	validation_0-auc:0.997693
[15]	validation_0-auc:0.997687
[16]	validation_0-auc:0.997895
[17]	validation_0-auc:0.997909
[18]	validation_0-auc:0.997916
[19]	validation_0-auc:0.998041
[20]	validation_0-auc:0.998089
[21]	validation_0-auc:0.998244
[22]	validation_0-auc:0.998287
[23]	validation_0-auc:0.998409
[24]	validation_0-auc:0.998399
[25]	validation_0-auc:0.99855
[26]	validation_0-auc:0.998505
[27]	validation_0-auc:0.998498
[28]	validation_0-auc:0.998543
[29]	validation_0-auc:0.998594
[30]	validation_0-auc:0.99863
[31]	validation_0-auc:0.998685
[32]	validation_0-auc:0

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s


[0]	validation_0-auc:0.961183
[1]	validation_0-auc:0.992493
[2]	validation_0-auc:0.995405
[3]	validation_0-auc:0.995473
[4]	validation_0-auc:0.995111
[5]	validation_0-auc:0.995415
[6]	validation_0-auc:0.996425
[7]	validation_0-auc:0.996419
[8]	validation_0-auc:0.997047
[9]	validation_0-auc:0.997038
[10]	validation_0-auc:0.997317
[11]	validation_0-auc:0.997578
[12]	validation_0-auc:0.997627
[13]	validation_0-auc:0.997755
[14]	validation_0-auc:0.997742
[15]	validation_0-auc:0.997828
[16]	validation_0-auc:0.998032
[17]	validation_0-auc:0.99804
[18]	validation_0-auc:0.998105
[19]	validation_0-auc:0.998209
[20]	validation_0-auc:0.998236
[21]	validation_0-auc:0.99827
[22]	validation_0-auc:0.998364
[23]	validation_0-auc:0.998467
[24]	validation_0-auc:0.998474
[25]	validation_0-auc:0.998562
[26]	validation_0-auc:0.998511
[27]	validation_0-auc:0.998513
[28]	validation_0-auc:0.998567
[29]	validation_0-auc:0.998601
[30]	validation_0-auc:0.998603
[31]	validation_0-auc:0.998668
[32]	validation_0-au

...

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 36.8min finished


[0]	validation_0-auc:0.994505
[1]	validation_0-auc:0.99485
[2]	validation_0-auc:0.996583
[3]	validation_0-auc:0.9971
[4]	validation_0-auc:0.997555
[5]	validation_0-auc:0.997591
[6]	validation_0-auc:0.997807
[7]	validation_0-auc:0.998049
[8]	validation_0-auc:0.998096
[9]	validation_0-auc:0.998182
[10]	validation_0-auc:0.998218
[11]	validation_0-auc:0.998488
[12]	validation_0-auc:0.998718
[13]	validation_0-auc:0.99872
[14]	validation_0-auc:0.998782
[15]	validation_0-auc:0.998819
[16]	validation_0-auc:0.998853
[17]	validation_0-auc:0.998898
[18]	validation_0-auc:0.998962
[19]	validation_0-auc:0.998959
[20]	validation_0-auc:0.999003
[21]	validation_0-auc:0.999024
[22]	validation_0-auc:0.99902
[23]	validation_0-auc:0.999027
[24]	validation_0-auc:0.999036
[25]	validation_0-auc:0.999064
[26]	validation_0-auc:0.999072
[27]	validation_0-auc:0.999094
[28]	validation_0-auc:0.999093
[29]	validation_0-auc:0.999088
[30]	validation_0-auc:0.999093
[31]	validation_0-auc:0.999101
[32]	validation_0-auc:0

[261]	validation_0-auc:0.999958
[262]	validation_0-auc:0.999958
[263]	validation_0-auc:0.999958
[264]	validation_0-auc:0.999958
[265]	validation_0-auc:0.999958
[266]	validation_0-auc:0.999958
[267]	validation_0-auc:0.999959
[268]	validation_0-auc:0.999959
[269]	validation_0-auc:0.999959
[270]	validation_0-auc:0.999959
[271]	validation_0-auc:0.999959
[272]	validation_0-auc:0.999959
[273]	validation_0-auc:0.999959
[274]	validation_0-auc:0.999959
[275]	validation_0-auc:0.999959
[276]	validation_0-auc:0.99996
[277]	validation_0-auc:0.99996
[278]	validation_0-auc:0.999961
[279]	validation_0-auc:0.999961
[280]	validation_0-auc:0.999961
[281]	validation_0-auc:0.99996
[282]	validation_0-auc:0.999961
[283]	validation_0-auc:0.999962
[284]	validation_0-auc:0.999962
[285]	validation_0-auc:0.999962
[286]	validation_0-auc:0.999962
[287]	validation_0-auc:0.999963
[288]	validation_0-auc:0.999963
[289]	validation_0-auc:0.999963
[290]	validation_0-auc:0.999963
[291]	validation_0-auc:0.999963
[292]	valid

[220]	validation_0-auc:0.999948
[221]	validation_0-auc:0.999949
[222]	validation_0-auc:0.999949
[223]	validation_0-auc:0.999949
[224]	validation_0-auc:0.999949
[225]	validation_0-auc:0.999949
[226]	validation_0-auc:0.99995
[227]	validation_0-auc:0.99995
[228]	validation_0-auc:0.99995
[229]	validation_0-auc:0.999951
[230]	validation_0-auc:0.999951
[231]	validation_0-auc:0.999952
[232]	validation_0-auc:0.999952
[233]	validation_0-auc:0.999953
[234]	validation_0-auc:0.999953
[235]	validation_0-auc:0.999954
[236]	validation_0-auc:0.999954
[237]	validation_0-auc:0.999954
[238]	validation_0-auc:0.999955
[239]	validation_0-auc:0.999955
[240]	validation_0-auc:0.999955
[241]	validation_0-auc:0.999954
[242]	validation_0-auc:0.999955
[243]	validation_0-auc:0.999955
[244]	validation_0-auc:0.999955
[245]	validation_0-auc:0.999955
[246]	validation_0-auc:0.999955
[247]	validation_0-auc:0.999956
[248]	validation_0-auc:0.999956
[249]	validation_0-auc:0.999956
[250]	validation_0-auc:0.999956
[251]	valid

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
       colsample_bytree=1.0, gamma=0, learning_rate=0.08, max_delta_step=0,
       max_depth=6, min_child_weight=10, missing=None, n_estimators=300,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [29]:
### XGBoost - PERFECT PARAMETERS FROM ABOVE

### prepare xgboost parameters
xgb_params = {
    'base_score':0.5, 
    'booster':'gbtree', 
    'colsample_bylevel':1.0,
    'colsample_bytree':1.0, 
    'gamma':0, 
    'learning_rate':0.08, 
    'max_delta_step':0,
    'max_depth':6, 
    'min_child_weight':10, 
    #'missing':None, 
    'n_estimators':300,
    'n_jobs':1, 
    'nthread':1, 
    'objective':'binary:logistic', 
    'random_state':0,
    'reg_alpha':0, 
    'reg_lambda':1, 
    'scale_pos_weight':1, 
    #'seed':None,
    'silent':True, 
    'subsample':0.8
}

dtrain = xgb.DMatrix(X_train, y_train)

dtest = xgb.DMatrix(X_test)

num_boost_rounds = 200
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred1 = model.predict(dtest)


print(f'The mean absolute error using the original training data set is {mean_absolute_error(xgb_pred1, y_test)}')
print(f'The mean squared error using the original training data set is {mean_squared_error(xgb_pred1, y_test)}')

#parameter tuning documentation: https://xgboost.readthedocs.io/en/latest/parameter.html

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


The mean absolute error using the original training data set is 0.006584542924764918
The mean squared error using the original training data set is 0.002034529507707721


In [26]:
### XGBoost 

### prepare xgboost parameters
xgb_params = {
    'objective': 'binary:logistic' ,
    #'max_depth':5,
    'gamma':0.1,
    'min_child_weight':5,
    'colsample_bytree':0.05,
    'eta':0.05
}

dtrain = xgb.DMatrix(X_train, y_train)

dtest = xgb.DMatrix(X_test)

num_boost_rounds = 200
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred1 = model.predict(dtest)


print(f'The mean absolute error using the original training data set is {mean_absolute_error(xgb_pred1, y_test)}')
print(f'The mean squared error using the original training data set is {mean_squared_error(xgb_pred1, y_test)}')

#parameter tuning documentation: https://xgboost.readthedocs.io/en/latest/parameter.html

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


The mean absolute error using the original training data set is 0.25126005223657316
The mean squared error using the original training data set is 0.0879454368450934


In [8]:
xgb_pred1

array([1.0000000e+00, 1.8101639e-09, 1.0000000e+00, ..., 7.6351876e-05,
       1.0000000e+00, 1.2987060e-04], dtype=float32)

In [9]:
y_test.head()

79479     1
26934     0
576749    1
385889    1
140926    1
Name: state, dtype: int64

In [10]:
def get_classification(predictions,threshold):
    classes = np.zeros_like(xgb_pred1)
    for i in range(len(classes)):
        if predictions[i] > threshold:
            classes[i] = 1
    return classes

In [11]:
import numpy as np
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,get_classification(xgb_pred1 ,0.5))
tn, fp, fn, tp = confusion_matrix(y_test,get_classification(xgb_pred1,0.5)).ravel()
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)


In [12]:
print(tpr, fpr)

0.9991578760748161 2.828694274722788e-05
